In [13]:
import pandas as pd
df = pd.read_csv("synth_power_mosfet.csv")
ioff_limit = 2000  # nA (default power_mainstream). Adjust if you changed PRESET
USABLE_RADIUS_MM = 147
# identify Ioff failures
ioff_fails = df[df["Ioff_nA"] > ioff_limit]
ioff_passes = df[df["Ioff_nA"] <= ioff_limit]

len_total = len(df)
len_ioff_fails = len(ioff_fails)
print(f"Ioff failures: {len_ioff_fails}/{len_total} = {100*len_ioff_fails/len_total:.2f}%")

# fraction of Ioff fails that have a physical defect
phys_count = ioff_fails["PhysicalDefect"].notna().sum()
print(f"Ioff fails with a physical defect: {phys_count} ({phys_count/len_ioff_fails:.2%})")
# breakdown by defect subtype
print(ioff_fails["PhysicalDefect"].value_counts())

#Pareto:
#print(df[df["Test Result"]=="Fail"]["FailureCause_First"].value_counts().head(20))
# chemical lot association
print(df.groupby("Chemical Lot ID")["Ioff_nA"].agg(["count","mean","median"]).sort_values("mean", ascending=False).head(10))
# slab: how many ioff fails per tool
print(df[df["Ioff_nA"]>ioff_limit].groupby("Tool ID").size().sort_values(ascending=False))
# probe card association
print(df[df["Ioff_nA"]>ioff_limit]["Probe Card ID"].value_counts().head(10))

# per wafer map counts
print(df[df["Ioff_nA"]>ioff_limit].groupby("Wafer ID").size().sort_values(ascending=False).head(10))

# compute fraction of Ioff fails that are near edges:
edge_frac = (ioff_fails["Defect Radius (mm)"] > (USABLE_RADIUS_MM * 0.85)).mean()
print("Fraction of Ioff fails at wafer edge:", edge_frac)

# correlation between Ioff and Vth or Idss
print(df[["Ioff_nA","Threshold Voltage (V)","Idss_A","On-Resistance (mΩ)"]].corr()["Ioff_nA"].sort_values())

#Possible temperature
print("Mean ambient temp for Ioff fails:", ioff_fails["Ambient Temperature (°C)"].mean())
print("Mean ambient temp for passes:", ioff_passes["Ambient Temperature (°C)"].mean())


Ioff failures: 52/517 = 10.06%
Ioff fails with a physical defect: 4 (7.69%)
PhysicalDefect
EdgeDelamination         2
ParticleContamination    1
EdgeCrack                1
Name: count, dtype: int64
                 count         mean      median
Chemical Lot ID                                
CL225              517  1025.928997  624.415761
Tool ID
P1    52
dtype: int64
Probe Card ID
PC-1006    52
Name: count, dtype: int64
Wafer ID
812c410f    52
dtype: int64
Fraction of Ioff fails at wafer edge: 0.19230769230769232
Threshold Voltage (V)    0.019669
On-Resistance (mΩ)       0.078213
Idss_A                   0.887912
Ioff_nA                  1.000000
Name: Ioff_nA, dtype: float64


In [15]:
# assuming df_all DataFrame
import numpy as np
# convert to A for Idss and Ioff (Ioff_nA -> A)
df["Ioff_A"] = df["Ioff_nA"] * 1e-9
# compute correlations in log space (avoid log(0))
eps = 1e-15
log_idss = np.log10(df["Idss_A"].clip(lower=eps))
log_ioff = np.log10(df["Ioff_A"].clip(lower=eps))

pearson = np.corrcoef(log_idss, log_ioff)[0,1]
print("Pearson corr (log10 space) between Idss and Ioff:", pearson)

# Break down by defect type
for dtype, sub in df.groupby("Defect Type"):
    if len(sub) >= 10:
        li = np.corrcoef(np.log10(sub["Idss_A"].clip(lower=eps)), np.log10(sub["Ioff_A"].clip(lower=eps)))[0,1]
        print(dtype, "corr (log10):", li, "n=", len(sub))


Pearson corr (log10 space) between Idss and Ioff: 0.6444464726232508
Parametric corr (log10): 0.31417307617303303 n= 50
Physical corr (log10): 0.6649333565170324 n= 13


In [16]:
# quick stats
df["Ioff_A"] = df["Ioff_nA"] * 1e-9
df["log_ioff"] = np.log10(df["Ioff_A"].clip(lower=1e-15))
df["log_idss"] = np.log10(df["Idss_A"].clip(lower=1e-15))

print("Ioff percentiles (nA):")
print(df["Ioff_nA"].quantile([0.5,0.9,0.95,0.99]))
print("Idss (A) percentiles:")
print(df["Idss_A"].quantile([0.5,0.9,0.95,0.99]))

print("Corr log10(Idss), log10(Ioff):", df[["log_idss","log_ioff"]].corr().iloc[0,1])

# Fraction of Ioff fails that are physical:
ioff_limit = 2000.0  # nA in your default
ioff_fails = df[df["Ioff_nA"] > ioff_limit]
print("Ioff fail count:", len(ioff_fails))
print("Fraction with PhysicalDefect:", ioff_fails["PhysicalDefect"].notna().mean())
print("Top physical types among Ioff fails:")
print(ioff_fails["PhysicalDefect"].value_counts().head())


Ioff percentiles (nA):
0.50     624.415761
0.90    2022.199698
0.95    2593.805243
0.99    4903.202708
Name: Ioff_nA, dtype: float64
Idss (A) percentiles:
0.50    2.766094e-07
0.90    5.969821e-07
0.95    7.988762e-07
0.99    1.379203e-06
Name: Idss_A, dtype: float64
Corr log10(Idss), log10(Ioff): 0.6444464726232497
Ioff fail count: 52
Fraction with PhysicalDefect: 0.07692307692307693
Top physical types among Ioff fails:
PhysicalDefect
EdgeDelamination         2
ParticleContamination    1
EdgeCrack                1
Name: count, dtype: int64
